# 1. Faiss-gpu 설치

In [1]:
pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85.5 MB 87 kB/s 


# 2. 데이터 프로세싱을 위한 준비

In [2]:
from ast import literal_eval
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
books = pd.read_csv('/content/drive/MyDrive/nlbooks.csv', encoding='utf-8')
books = books.replace({np.nan: 'none'})

books_df=books[['title','description']]

#books_df[['description']=='none']=None
books_df_nax=books_df.dropna(axis='rows')


from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


cnt_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
tfidf = TfidfVectorizer(stop_words='english')
# fit_transform안에 데이터프레임형태로 넣어주면 안됨. 하나의 변수씩만 넣어주자!
title_vect1 = cnt_vect.fit_transform(books_df_nax['title'])
title_vect2 = tfidf.fit_transform(books_df_nax['title'])
#des_vect = cnt_vect.fit_transform(books['description'])
#books['book_info']=books['title']+books['description']
#b_category=books['kdc_class_no']

# 책 제목에 따른 코사인 유사도 추출
#title_sim = cosine_similarity(title_vect, title_vect)
#des_sim=cosine_similarity(des_vect, des_vect)
#mix_sim=title_sim + des_sim
#print(mix_sim[:6])
#print(title_sim[:6])

In [ ]:
#conda install faiss-gpu -c pytorch # [DEFAULT]For CUDA8.0, comes with cudatoolkit8.0
#conda install faiss-gpu cuda90 -c pytorch # For CUDA9.0
# conda install faiss-gpu cuda91 -c pytorch # For CUDA9.1
# cuda90/cuda91 shown above is a feature, it doesn't install CUDA for you.

# 3. 희소행렬 차원축소 TruncatedSVD

In [3]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
import numpy as np

X = title_vect2
svd = TruncatedSVD(n_components=500, n_iter=7, random_state=42)
X_new=svd.fit_transform(X)

In [ ]:
X_new.shape

(157118, 500)

In [ ]:
#des_vect = tfidf.fit_transform(books_df_nax['description'])
#svd = TruncatedSVD(n_components=500, n_iter=7, random_state=42)
#new_des_vect=svd.fit_transform(des_vect)

In [4]:
import faiss
import numpy as np

D = X_new.shape[1]
K = 500  # The number of clusters
X = X_new.astype(np.float32)


# Setup
kmeans = faiss.Kmeans(d=D, k=K, niter=20, verbose=True,gpu=True)
# For GPU(s), run the following line. This will use all GPUs
# kmeans = faiss.Kmeans(d=D, k=K, niter=20, verbose=True, gpu=True)

# Run clustering
kmeans.train(X)

# Error for each iteration
print(kmeans.obj)  # array with 20 elements

# Centroids after clustering
print(kmeans.centroids.shape)  # (10, 500)

# The assignment for each vector.
dists, ids = kmeans.index.search(X, 1)  # Need to run NN search again
print(ids.shape)  # (157118, 1)

# Params
print("D:", kmeans.d)
print("K:", kmeans.k)
print("niter:", kmeans.cp.niter)

[13895.7890625  10826.734375    9281.20214844  8693.04101562
  8352.88378906  8137.22900391  8009.79833984  7902.28613281
  7835.91552734  7773.09082031  7699.23144531  7642.52148438
  7595.60595703  7557.33349609  7530.17285156  7510.10302734
  7499.12060547  7484.41259766  7473.23925781  7467.47851562]
(500, 500)
(157118, 1)
D: 500
K: 500
niter: 20


In [5]:
ids

array([[292],
       [292],
       [111],
       ...,
       [136],
       [ 31],
       [ 12]])

In [6]:
books_df_nax['tit_cluster']=ids

In [ ]:
books_df_nax[books_df_nax['tit_cluster']==292]

,title,description,tit_cluster
0,너에게 목소리를 보낼게 - &lt;달빛천사&gt; 성우 이용신의 첫 번째 에세이,2004년 방영한 애니메이션 &lt;달빛천사&gt;에서 주인공 루나(풀문) 역을 맡...,292
1,일기에도 거짓말을 쓰는 사람 - 99년생 시인의 자의식 과잉 에세이,“그러니 나는 말하고 싶은 것을 말하겠다”「침착하게 사랑하기」 차도하 시인 첫 에세...,292
3280,"오늘도, 소리도 - 이치운의 바다 에세이",수필 작가로서의 내공은 단일 소재에서 다층적인 이미지와 주제를 갈라 내는 우무로 알...,292
3320,"이타적 개인주의자 - 선 긋는데 선수이며, 남 돕는데 산수인 그가 혼자 커피 마시며...",저자 김인호의『이타적 개인주의자』은 크게 6장으로 나누어져 있으며 주옥같은 작품을 ...,292
3322,진짜 어른이 된다는 것은 - 일상에서 나만의 행복을 찾아가는 진솔하고 투명한 성찰 에세이,저자의 세 번째 에세이집. 지난 한 해 동안 책 읽고 산책하면서 나를 돌아보며 성찰...,292
...,...,...,...
152681,에세이 손자병법,none,292
153372,영어시간에 매를 맞던 소년이 쓰는 영어 에세이,none,292
155364,"어쩌다, 제주 - 서울 여자의 제주도 마실 에세이",서울 사는 회화 작가의 제주도 마실 에세이. 회화 작가인 저자는 2015년부터 제주...,292
156456,투자의 민낯 - 본격 주식투자 뒷담화 에세이,이제 성공한 5%의 이야기가 아닌 95%의 이야기를 들을 시간. &lt오마이뉴스&g...,292


In [ ]:
books_df_nax[books_df_nax['tit_cluster']==31]

,title,description,tit_cluster
15166,(한권으로 끝내는)관광중국어,이 책은 관광통역안내원이 갖추어야 할 기본적인 소양과 자세를 제시하고 이들이 해야 ...,31
25139,(한권으로 끝내는)중국어작문,본 교재는 중국어와 한국어가 지닌 문법적 특성을 대조 분석함으로써 두 언어 사이의 ...,31
25795,(한권으로 끝내는)땡땡땡 초등교육과정,제7차 해설서 내용을 중심으로 지도서의 총론과 교과 교육론 핵심을 요약해 실었다. ...,31
29683,(수석합격생 간호사 안미영의) 한권으로 끝내는 공인중개사:2차대비,2006년 17회 공인중개사 2차 시험 대비 수험서. 처음 공부하는 수험생들의 기초...,31
29685,(국가공인) 신용관리사:한권으로 끝내는,신용관리사 자격 시험 대비 수험서. 이 책은 법률에 대한 기본지식이 없는 일반인들이...,31
...,...,...,...
147194,간호사 면접 기출문제 해설 :한권으로 끝내는 면접의 神,간호사 면접 기출문제를 해설하는 수험서다. 실제 면접을 보는 방식으로 구성하였다. ...,31
152498,(한권으로 만나는) 세계문학 50선,none,31
154497,2022 기술직공무원 식품위생직 식품화학 한권으로 끝내기 - 9급 지방직ㆍ교육청 채...,총 11개의 단원으로 구성된 이론이 체계적으로 꼼꼼하게 정리되어 있다. 시험에 나오...,31
154511,2022 조경기사 필기 한권으로 끝내기 - 2021년 최근 기출문제와 해설 수록! ...,“핵심이론+적중예상문제+기출문제” 3단계로 구분해 한 권으로 엮음으로써 최단기간에 ...,31


In [ ]:
books_df_nax[books_df_nax['tit_cluster']==111]

,title,description,tit_cluster
2,본격 한중일 세계사 12 - 임오군란과 통킹 위기,한중일 관계의 결정적 분기점인 임오군란의 막전 막후를 다룬다. 러시아의 세력 확장을...,111
1012,이상한 아빠,none,111
1521,도쿄,none,111
1800,이상한 안경,none,111
2701,이상한 과자 가게 전천당 12,none,111
...,...,...,...
153344,어느 날,none,111
153417,이상한 꿈,none,111
153418,이상한 서평단의 이상한 서평,none,111
153419,이상한 휴가,none,111


In [ ]:
books_df_nax[books_df_nax['tit_cluster']==12]

,title,description,tit_cluster
22,권력의 미래 - 소프트 파워 리더십은 어떻게 세상을 바꾸는가？,"조지프 나이가 다양한 활동 끝에 권력적 관계의 핵심 요소들을 분석한 책으로, 서문에...",12
629,"예술은 어떻게 비즈니스의 무기가 되는가 - 0에서 1을 창조하는 혁신적 사고법, 아...",아트 씽킹을 키우기 위한 다양한 훈련법이 나온다. 그림을 그리며 아이디어를 확장하는...,12
689,독서지도 어떻게 할 것인가 . 1,none,12
768,주제는 어떻게 정할까?. 3,none,12
797,고대 이집트 사람들은 어떻게 살았을까?,none,12
...,...,...,...
155361,평범한 직장인이 어떻게 1년 만에 2권의 책을 썼을까,저자는 작가가 되겠다는 마음을 먹은 지 7개월 만에 마침내 작가가 되었다. 이 책은...,12
155594,남자의 품격 - 중세의 기사는 어떻게 남자로 만들어졌는가,남성다움의 전형으로 일컬어지는 중세의 기사는 어떻게 만들어졌을까? 저자는 플랑드르 ...,12
155636,화학공학기술자 어떻게 되었을까? - 현직 화학공학기술자들을 통해 알아보는 리얼 직업...,엄마 아빠도 모르고 선생님도 몰랐던 화학공학기술자 6인이 말하는 직업인의 이야기를 ...,12
156850,가족이야기는 어떻게 만들어지는가,가족 이데올로기는 우리 사회를 지배하는 대표적인 담론 중 하나이다. 사회 구성의 기...,12


In [7]:
import random

# user X의 book list (예시)
X=['에세이 손자병법','(한권으로 끝내는)관광중국어','이상한 꿈','고대 이집트 사람들은 어떻게 살았을까?'] # 사용자가 갖고 있는 책 리스트 데이터


# 사용자의 책 리스트를 input으로 받아 그 리스트 중 하나를 반환하는 함수
def userbooks(A):
    l=len(A)
    n=random.randrange(0,l)
    return A[n]

In [33]:
B=userbooks(X)
print(B)

이상한 꿈


In [8]:
n=books_df_nax.index[books_df_nax['title']=='에세이 손자병법'].tolist()
n

[152681]

In [10]:
n[0]

152681

In [ ]:
pip install sentence_transformers

In [ ]:
#Write some lines to encode (sentences 0 and 2 are both ideltical):
#sen = books_df_nax['description']
#from sentence_transformers import SentenceTransformer
#model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
##Encoding:
#sen_embeddings = model.encode(sen)
#sen_embeddings.shape

In [ ]:
def sim_idx_with_A(df,books):
    n=int(df[df['title']==books]['tit_cluster'])
    same_clu_books_df=df[df['tit_cluster']==n]
    sen = same_clu_books_df['description']

    # 1번 후보 : sentence transformer 유사도 구하기
    #from sentence_transformers import SentenceTransformer
    #model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    #sen_embeddings = model.encode(sen)
    
    # 2번 후보 : 단순 tfidf 활용한 유사도 구하기
    clu_tfidf = tfidf.fit_transform(sen)
    clu_des_sim=cosine_similarity(clu_tfidf,clu_tfidf)

    idx_list=df.index[df['title']==books].tolist()
    idx=idx_list[0]
    books_sim_vect=clu_des_sim[idx:idx+1]
    books_des_sim_idx = books_sim_vect.argsort()[::-1]

    return books_des_sim_idx

# 사용자 책 리스트에서 랜덤 추출한 책 기준 유사한 책 인덱스 리스트 반환
sim_books_idx=sim_idx_with_A(books_df_nax,B)

In [ ]:
def find_sim_books(df,books,top_n):
    #books=userbooks(A)
    #n=int(df[df['title']==books]['tit_cluster'])
    #same_clu_books_df=df[df['tit_cluster']==n]
    #sen = same_clu_books_df['description']
    sim_books_idx=sim_idx_with_A(df,books)
    top_sim_idx=sim_books_idx[:top_n]
    top_sim_idx=top_sim_idx.reshape(-1,)
    sim_books=df.iloc[top_sim_idx]

    return sim_books

In [ ]:
des_sim_books=find_sim_books(books_df_nax,B,5) # B 책과 des 비슷한 상위 5개 책 데이터프레임

In [15]:
import pandas as pd
a=pd.DataFrame([[1,14,3,20,0],
   [21,11,4,5,10],
   [6,4,8,7,10]])
n=a[a[1]==0][2]

a

,0,1,2,3,4
0,1,14,3,20,0
1,21,11,4,5,10
2,6,4,8,7,10


In [22]:
a[1:2]

,0,1,2,3,4
1,21,11,4,5,10


In [14]:
import numpy as np
import pandas as pd
a=pd.DataFrame({"A":[1,14,3,20,0],
   "B":[21,11,4,5,10],
   "C":[6,4,8,7,10]})
a
#a_idx=np.argsort(a)
#a_idx


,A,B,C
0,1,21,6
1,14,11,4
2,3,4,8
3,20,5,7
4,0,10,10


In [ ]:
a = np.array([1,14,3,20,0])
s = a.argsort()[::-1]
print(s)

[3 1 2 0 4]


In [32]:
import numpy as np

a=np.array([1,14,3,20,0])
s=a.argsort()[::-1][:2]
print(s)
print(a[s])
sr=s.reshape(-1,)
print(sr)

[3 1]
[20 14]
[3 1]


In [31]:
a = np.array([1,14,3,20,0])
s = a.argsort()[::-1]
print(s) #인덱스값 출력
print(a[s]) #원소값출력
sr=s.reshape(-1,)
print(sr)

[3 1 2 0 4]
[20 14  3  1  0]
[3 1 2 0 4]


In [ ]:
import numpy as np
a=np.array([[1,14,3,20,0],
            [21,11,4,5,10],
            [6,4,8,7,10]])
s=np.argsort(a)
print(s)

[[4 0 2 1 3]
 [2 3 4 1 0]
 [1 0 3 2 4]]


In [ ]:
pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.4 MB/s 
     |████████████████████████████████| 5.5 MB 44.1 MB/s 
     |████████████████████████████████| 1.3 MB 57.6 MB/s 
     |████████████████████████████████| 163 kB 69.9 MB/s 
     |████████████████████████████████| 7.6 MB 52.9 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=0527edb765c0add00be81d520b37408c477cc2a99698cfcf2ee9aa99fe56ba4f
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [ ]:
#Write some lines to encode (sentences 0 and 2 are both ideltical):
sen = books_df_nax['description']
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
#Encoding:
sen_embeddings = model.encode(sen)
sen_embeddings.shape

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

KeyboardInterrupt: ignored

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
des_sim=cosine_similarity(clu1_tfidf,clu1_tfidf)